#   Airbnb Data Analysis Report


###   Prepare and Download DataSet


In [6]:
#  initial directorys
!mkdir data/
!mkdir data/boston
!mkdir data/seattle

#  download data from oss
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/seattle/calendar.csv -O data/seattle/calendar.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/seattle/listings.csv -O data/seattle/listings.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/seattle/reviews.csv -O data/seattle/reviews.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/boston/calendar.csv -O data/boston/calendar.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/boston/listings.csv -O data/boston/listings.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/boston/reviews.csv -O data/boston/reviews.csv



--2018-09-05 12:12:39--  https://oss.hacking-linux.com/udacity/airbnb/seattle/calendar.csv
Resolving oss.hacking-linux.com (oss.hacking-linux.com)... 47.75.18.176, 47.75.18.190
Connecting to oss.hacking-linux.com (oss.hacking-linux.com)|47.75.18.176|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 36650955 (35M) [text/csv]
Saving to: 'data/seattle/calendar.csv'

data/seattle/calend 100%[===================>]  34.95M  11.1MB/s    in 3.2s    

2018-09-05 12:12:43 (11.1 MB/s) - 'data/seattle/calendar.csv' saved [36650955/36650955]

--2018-09-05 12:12:43--  https://oss.hacking-linux.com/udacity/airbnb/seattle/listings.csv
Resolving oss.hacking-linux.com (oss.hacking-linux.com)... 47.75.18.176, 47.75.18.190
Connecting to oss.hacking-linux.com (oss.hacking-linux.com)|47.75.18.176|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 16871061 (16M)